# Leveraging pre-trained APIs in Amazon Comprehend

## Table of contents
- [Introduction](#intro)
- [Setup](#setup)
- [Identifying Named Entities](#identifying-named-entities)
- [Detecting Key Phrases](#detecting-key-phrases)
- [Identifying the Dominant Language](#identifying-the-ominant-language)
- [Determining Emotional sentiment](#determining-emotional-sentiment)
- [Determining Syntax](#determiningsyntax)
- [Detecting Personally Identifiable Information (PII)](#detecting-pii)
- [Conclusion](#conclusion)
- [Clean Up](#clean-up)

## <a name='intro' ></a>Introduction

This notebook provides step-by-step instructions to use the [Amazon Comprehend](https://aws.amazon.com/comprehend/)'s pre-trained APIs to to uncover information in unstructured data. Amazon Comprehend uses a pre-trained model to examine and analyze a document or set of documents to gather insights about it. This model is continuously trained on a large body of text so that there is no need for you to provide training data.

We will explore 6 pre-trained APIs: Identifying Named Entities, Extracting Key Phrases, Identifying the Dominant Language, Determining Emotional sentiment, Determining Syntax, Detecting Detect Personally Identifiable Information (PII).

This Notebook uses AWS resources and you may incur a cost when running these cells.

## <a name="tips" ></a>Tips

If you are new to Python Notebooks: `SHIFT` + `ENTER` will execute a code cell and go to the next one.

## <a name='setup' ></a>Setup
We import the relevant packages to interact with Amazon Comprehend. [Boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) is the AWS Python SDK.

In [1]:
import boto3

We specify the SageMaker execution role, this is the role that is used in this notebook and the region the notebook is in.

In [2]:
import sagemaker
from sagemaker import get_execution_role
role = get_execution_role()
region = boto3.Session().region_name

We import other packages we will use.

In [3]:
import json
import pandas as pd
import numpy as np

## Starting the Amazon Comprehend client

In [4]:
comprehend = boto3.client(service_name='comprehend', region_name=region)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


## Data

In this lab, we will use the same sample input text used in the [Amazon Comprehend console](https://console.aws.amazon.com/comprehend). We have copyed it here for convenience. If you wish to experiment with a different text, simply modify the cell bellow.

In [5]:
sample_text = '''
Hello Zhang Wei. Your AnyCompany Financial Services, LLC credit card account 1111-0000-1111-0000 has a minimum payment of $24.53 that is due by July 31st. Based on your autopay settings, we will withdraw your payment on the due date from your bank account XXXXXX1111 with the routing number XXXXX0000. 
Your latest statement was mailed to 100 Main Street, Anytown, WA 98121. 
After your payment is received, you will receive a confirmation text message at 206-555-0100. 
If you have questions about your bill, AnyCompany Customer Service is available by phone at 206-555-0199 or email at support@anycompany.com.
'''

## <a name='identifying-named-entities' ></a>Identifying Named Entities

A named entity is a real-world object (persons, places, locations, organizations, etc.) that can be denoted with a proper name.

Amazon Comprehend can extract named entities from a document or text. This can be useful, for example, for indexing, document labeling or search. For more information, see [Detect Entities](https://docs.aws.amazon.com/comprehend/latest/dg/API_DetectEntities.html)).

The API used to extract these entities is the [DetectEntities API](https://docs.aws.amazon.com/comprehend/latest/dg/API_DetectEntities).

For each entity detected Amazon Comprehend returns both the type, for instance "Person" or "Date", as well as a confidence score which indicates how confident the model is in this detection. In your implementation you can use this confidence score to set threshold values.

In [6]:
print('Calling DetectEntities')
detected_entities = comprehend.detect_entities(Text=sample_text, LanguageCode='en')
print(json.dumps(detected_entities, sort_keys=True, indent=4))
print('End of DetectEntities\n')

Calling DetectEntities
{
    "Entities": [
        {
            "BeginOffset": 7,
            "EndOffset": 16,
            "Score": 0.9993807077407837,
            "Text": "Zhang Wei",
            "Type": "PERSON"
        },
        {
            "BeginOffset": 23,
            "EndOffset": 57,
            "Score": 0.9996546506881714,
            "Text": "AnyCompany Financial Services, LLC",
            "Type": "ORGANIZATION"
        },
        {
            "BeginOffset": 78,
            "EndOffset": 97,
            "Score": 0.9905166029930115,
            "Text": "1111-0000-1111-0000",
            "Type": "OTHER"
        },
        {
            "BeginOffset": 123,
            "EndOffset": 129,
            "Score": 0.9989154934883118,
            "Text": "$24.53",
            "Type": "QUANTITY"
        },
        {
            "BeginOffset": 145,
            "EndOffset": 154,
            "Score": 0.9994641542434692,
            "Text": "July 31st",
            "Type": "DATE"
        

The response includes the full score, type, and offsets.

Now lets make it a bit more human readable:

In [7]:
detectec_entities_df = pd.DataFrame([ [entity['Text'], entity['Type'], entity['Score']] for entity in detected_entities['Entities']],
                columns=['Text', 'Type', 'Score'])

print('This was the text analyzed:')
print(sample_text)
print()
display (detectec_entities_df)

This was the text analyzed:

Hello Zhang Wei. Your AnyCompany Financial Services, LLC credit card account 1111-0000-1111-0000 has a minimum payment of $24.53 that is due by July 31st. Based on your autopay settings, we will withdraw your payment on the due date from your bank account XXXXXX1111 with the routing number XXXXX0000. 
Your latest statement was mailed to 100 Main Street, Anytown, WA 98121. 
After your payment is received, you will receive a confirmation text message at 206-555-0100. 
If you have questions about your bill, AnyCompany Customer Service is available by phone at 206-555-0199 or email at support@anycompany.com.




,Text,Type,Score
0,Zhang Wei,PERSON,0.999381
1,"AnyCompany Financial Services, LLC",ORGANIZATION,0.999655
2,1111-0000-1111-0000,OTHER,0.990517
3,$24.53,QUANTITY,0.998915
4,July 31st,DATE,0.999464
5,XXXXXX1111,OTHER,0.957768
6,XXXXX0000,OTHER,0.954860
7,"100 Main Street, Anytown, WA 98121",LOCATION,0.989941
8,206-555-0100,OTHER,0.999637
9,AnyCompany Customer,ORGANIZATION,0.801927


## <a name='detecting-key-phrases' ></a>Detecting Key Phrases

Amazon Comprehend can extract key noun phrases that appear in a document. For example, a document about a basketball game might return the names of the teams, the name of the venue, and the final score. This can be used, for example, for indexing or summarization. For more information, see [Detect Key Phrases](https://docs.aws.amazon.com/comprehend/latest/dg/get-started-api-key-phrases.html).

The API used to extract these key phrases is the [DetectKeyPhrases API](https://docs.aws.amazon.com/comprehend/latest/dg/API_DetectKeyPhrases).

Amazon Comprehend returns the key phrases, as well as a confidence score which indicates how confident the model is in this detection. In your implementation you can use this confidence score to set threshold values.

In [8]:
print('Calling DetectKeyPhrases')
detected_key_phrases = comprehend.detect_key_phrases(Text=sample_text, LanguageCode='en')
print(json.dumps(detected_key_phrases, sort_keys=True, indent=4))
print('End of DetectKeyPhrases\n')

Calling DetectKeyPhrases
{
    "KeyPhrases": [
        {
            "BeginOffset": 1,
            "EndOffset": 16,
            "Score": 0.8426344990730286,
            "Text": "Hello Zhang Wei"
        },
        {
            "BeginOffset": 18,
            "EndOffset": 52,
            "Score": 0.9881375432014465,
            "Text": "Your AnyCompany Financial Services"
        },
        {
            "BeginOffset": 54,
            "EndOffset": 97,
            "Score": 0.8444651961326599,
            "Text": "LLC credit card account 1111-0000-1111-0000"
        },
        {
            "BeginOffset": 102,
            "EndOffset": 119,
            "Score": 0.9999476075172424,
            "Text": "a minimum payment"
        },
        {
            "BeginOffset": 123,
            "EndOffset": 129,
            "Score": 0.9997788071632385,
            "Text": "$24.53"
        },
        {
            "BeginOffset": 145,
            "EndOffset": 154,
            "Score": 0.999465107917785

The response includes the full score, key phrase text, and offsets.

Now lets make it a bit more human readable:

In [9]:
detected_key_phrases_df = pd.DataFrame([ [entity['Text'], entity['Score']] for entity in detected_key_phrases['KeyPhrases']],
                columns=['Text', 'Score'])

print('This was the text analyzed:')
print(sample_text)
print()
display (detected_key_phrases_df)

This was the text analyzed:

Hello Zhang Wei. Your AnyCompany Financial Services, LLC credit card account 1111-0000-1111-0000 has a minimum payment of $24.53 that is due by July 31st. Based on your autopay settings, we will withdraw your payment on the due date from your bank account XXXXXX1111 with the routing number XXXXX0000. 
Your latest statement was mailed to 100 Main Street, Anytown, WA 98121. 
After your payment is received, you will receive a confirmation text message at 206-555-0100. 
If you have questions about your bill, AnyCompany Customer Service is available by phone at 206-555-0199 or email at support@anycompany.com.




,Text,Score
0,Hello Zhang Wei,0.842634
1,Your AnyCompany Financial Services,0.988138
2,LLC credit card account 1111-0000-1111-0000,0.844465
3,a minimum payment,0.999948
4,$24.53,0.999779
5,July 31st,0.999465
6,your autopay settings,0.999296
7,your payment,0.999452
8,the due date,0.999488
9,your bank account XXXXXX1111,0.984577


## <a name='identifyinf-the-dominant-language'></a> Identifying the Dominant Language

Amazon Comprehend identifies the dominant language in a document. Amazon Comprehend can currently identify many languages. This can be useful as a first step before further processing, for example when phone call transcripts can be in different languages. For more information, including which languages can be identified, see [Detect the Dominant Language](https://docs.aws.amazon.com/comprehend/latest/dg/how-languages.html).

The API used to identify the dominant language is the [DetectDominantLanguage API](https://docs.aws.amazon.com/comprehend/latest/dg/API_DetectDominantLanguage).

Amazon Comprehend returns the dominant language, as well as a confidence score which indicates how confident the model is in this detection. In your implementation you can use this confidence score to set threshold values. If more than one language is detected, it will return each detected language and its corresponding confidence score.

In [10]:
print('Calling DetectDominantLanguage')
detected_language = comprehend.detect_dominant_language(Text=sample_text)
print(json.dumps(detected_language, sort_keys=True, indent=4))
print('End of DetectDominantLanguage\n')

Calling DetectDominantLanguage
{
    "Languages": [
        {
            "LanguageCode": "en",
            "Score": 0.9913273453712463
        }
    ],
    "ResponseMetadata": {
        "HTTPHeaders": {
            "content-length": "64",
            "content-type": "application/x-amz-json-1.1",
            "date": "Wed, 25 May 2022 12:58:25 GMT",
            "x-amzn-requestid": "37c52fb3-30aa-4cc2-9f0f-64424319548b"
        },
        "HTTPStatusCode": 200,
        "RequestId": "37c52fb3-30aa-4cc2-9f0f-64424319548b",
        "RetryAttempts": 0
    }
}
End of DetectDominantLanguage



The response includes the full score, and the detected language codes.

Now lets make it a bit more human readable:

In [11]:
detected_language_df = pd.DataFrame([ [code['LanguageCode'], code['Score']] for code in detected_language['Languages']],
                columns=['Language Code', 'Score'])

print('This was the text analyzed:')
print(sample_text)
print()
display (detected_language_df)

This was the text analyzed:

Hello Zhang Wei. Your AnyCompany Financial Services, LLC credit card account 1111-0000-1111-0000 has a minimum payment of $24.53 that is due by July 31st. Based on your autopay settings, we will withdraw your payment on the due date from your bank account XXXXXX1111 with the routing number XXXXX0000. 
Your latest statement was mailed to 100 Main Street, Anytown, WA 98121. 
After your payment is received, you will receive a confirmation text message at 206-555-0100. 
If you have questions about your bill, AnyCompany Customer Service is available by phone at 206-555-0199 or email at support@anycompany.com.




,Language Code,Score
0,en,0.991327


## <a name='determining-emotional-sentiment' ></a>Determining Emotional Sentiment

Amazon Comprehend determines the emotional sentiment of a document. Sentiment can be positive, neutral, negative, or mixed. For more information, see Determine Sentiment. This can be useful for example to analyze the content of reviews or transcripts from call centres. For more information, see [Detecting Sentiment](https://docs.aws.amazon.com/comprehend/latest/dg/get-started-api-sentiment.html).

The API used to extract the emotional sentiment is the [DetectSentiment API](https://docs.aws.amazon.com/comprehend/latest/dg/API_DetectSentiment).

Amazon Comprehend returns the different sentiments and the related confidence score for each of them, which indicates how confident the model is in this detection. The sentiment with the highest confidence score can be seen as the predominant sentiment in the text.

In [12]:
print('Calling DetectSentiment')
detected_sentiment = comprehend.detect_sentiment(Text=sample_text, LanguageCode='en')
print(json.dumps(detected_sentiment, sort_keys=True, indent=4))
print('End of DetectSentiment\n')

Calling DetectSentiment
{
    "ResponseMetadata": {
        "HTTPHeaders": {
            "content-length": "163",
            "content-type": "application/x-amz-json-1.1",
            "date": "Wed, 25 May 2022 12:58:39 GMT",
            "x-amzn-requestid": "67d19a98-5db6-4e42-a68f-ef2ed4c6c54b"
        },
        "HTTPStatusCode": 200,
        "RequestId": "67d19a98-5db6-4e42-a68f-ef2ed4c6c54b",
        "RetryAttempts": 0
    },
    "Sentiment": "NEUTRAL",
    "SentimentScore": {
        "Mixed": 9.69591928878799e-06,
        "Negative": 0.016147520393133163,
        "Neutral": 0.9832557439804077,
        "Positive": 0.0005869901506230235
    }
}
End of DetectSentiment



The response includes the predominant sentiment and the full scores for each detected sentiment.

Now lets make it a bit more human readable:

In [13]:
predominant_sentiment = detected_sentiment['Sentiment']
detected_sentiments_df = pd.DataFrame([ [sentiment, detected_sentiment['SentimentScore'][sentiment]] for sentiment in detected_sentiment['SentimentScore']],
                columns=['Language Code', 'Score'])

print('This was the text analyzed:')
print(sample_text)
print()
print('The predominant sentiment is {}.'.format(predominant_sentiment))
print()
display (detected_sentiments_df)

This was the text analyzed:

Hello Zhang Wei. Your AnyCompany Financial Services, LLC credit card account 1111-0000-1111-0000 has a minimum payment of $24.53 that is due by July 31st. Based on your autopay settings, we will withdraw your payment on the due date from your bank account XXXXXX1111 with the routing number XXXXX0000. 
Your latest statement was mailed to 100 Main Street, Anytown, WA 98121. 
After your payment is received, you will receive a confirmation text message at 206-555-0100. 
If you have questions about your bill, AnyCompany Customer Service is available by phone at 206-555-0199 or email at support@anycompany.com.


The predominant sentiment is NEUTRAL.



,Language Code,Score
0,Positive,0.000587
1,Negative,0.016148
2,Neutral,0.983256
3,Mixed,0.000010


## <a name="determining-syntax" ></a> Determining Syntax

Amazon Comprehend parses each word in your document and determines the syntax, the part of speech, for the word. For example, in the sentence "It is raining today in Seattle," "it" is identified as a pronoun, "raining" is identified as a verb, and "Seattle" is identified as a proper noun. For more information, see [Analyze Syntax](https://docs.aws.amazon.com/comprehend/latest/dg/how-syntax.html).

The API used to extract thesyntax information is the [DetectSyntax API](https://docs.aws.amazon.com/comprehend/latest/dg/API_DetectSyntax).

Amazon Comprehend returns the different parts of speech and the related confidence score for each of them, which indicates how confident the model is in this detection, token Ids and offsets.

In [14]:
print('Calling DetectSyntax')
detected_syntax = comprehend.detect_syntax(Text=sample_text, LanguageCode='en')
print(json.dumps(detected_syntax, sort_keys=True, indent=4))
print('End of DetectSyntax\n')

Calling DetectSyntax
{
    "ResponseMetadata": {
        "HTTPHeaders": {
            "content-length": "12896",
            "content-type": "application/x-amz-json-1.1",
            "date": "Wed, 25 May 2022 12:59:03 GMT",
            "x-amzn-requestid": "2cd5e213-0a1d-40f4-8683-f3be777dece9"
        },
        "HTTPStatusCode": 200,
        "RequestId": "2cd5e213-0a1d-40f4-8683-f3be777dece9",
        "RetryAttempts": 0
    },
    "SyntaxTokens": [
        {
            "BeginOffset": 1,
            "EndOffset": 6,
            "PartOfSpeech": {
                "Score": 0.9888805150985718,
                "Tag": "INTJ"
            },
            "Text": "Hello",
            "TokenId": 1
        },
        {
            "BeginOffset": 7,
            "EndOffset": 12,
            "PartOfSpeech": {
                "Score": 0.9991546273231506,
                "Tag": "PROPN"
            },
            "Text": "Zhang",
            "TokenId": 2
        },
        {
            "BeginOffset": 1

Amazon Comprehend returns the text, different parts of speech, confidence score for each of them, token Ids and offsets.

Now lets make it a bit more human readable:

In [15]:
detected_syntax_df = pd.DataFrame([ [part['Text'], part['PartOfSpeech']['Tag'], part['PartOfSpeech']['Score']] for part in detected_syntax['SyntaxTokens']],
                columns=['Text', 'Part Of Speech', 'Score'])

print('This was the text analyzed:')
print(sample_text)
print()
print('First twenty tokens:')
display (detected_syntax_df.head(20))

This was the text analyzed:

Hello Zhang Wei. Your AnyCompany Financial Services, LLC credit card account 1111-0000-1111-0000 has a minimum payment of $24.53 that is due by July 31st. Based on your autopay settings, we will withdraw your payment on the due date from your bank account XXXXXX1111 with the routing number XXXXX0000. 
Your latest statement was mailed to 100 Main Street, Anytown, WA 98121. 
After your payment is received, you will receive a confirmation text message at 206-555-0100. 
If you have questions about your bill, AnyCompany Customer Service is available by phone at 206-555-0199 or email at support@anycompany.com.


First twenty tokens:


,Text,Part Of Speech,Score
0,Hello,INTJ,0.988881
1,Zhang,PROPN,0.999155
2,Wei,PROPN,0.998299
3,.,PUNCT,0.999998
4,Your,PRON,0.998580
5,AnyCompany,PROPN,0.992449
6,Financial,PROPN,0.998791
7,Services,PROPN,0.998121
8,",",PUNCT,0.999969
9,LLC,PROPN,0.985515


## <a name='detecting-pii' ></a> Detecting Personally Identifiable Information (PII)

Amazon Comprehend analyzes documents to detect personal data that could be used to identify an individual, such as an address, bank account number, or phone number. This can be usefull, for example, for information extraction and indexing, and to comply with legal requirements around data protection. For more information, see [Detect Personally Identifiable Information (PII)](https://docs.aws.amazon.com/comprehend/latest/dg/how-pii.html).

Amazon Comprehend can help you identify the location of individual PII in your document or help you label documents that contain PII. 

### Identify the location of PII in your text documents

Amazon Comprehend can help you identify the location of individual PII in your document. Select "Offsets" in the Personally identifiable information (PII) analysis mode.

The API used to identify the location of individual PII is the [DetectPiiEntities API](https://docs.aws.amazon.com/comprehend/latest/dg/API_DetectPiiEntities.html).

Amazon Comprehend returns the different PII and the related confidence score for each of them, which indicates how confident the model is in this detection. 

In [16]:
print('Calling DetectPiiEntities')
detected_pii_entities = comprehend.detect_pii_entities(Text=sample_text, LanguageCode='en')
print(json.dumps(detected_pii_entities, sort_keys=True, indent=4))
print('End of DetectPiiEntities\n')

Calling DetectPiiEntities
{
    "Entities": [
        {
            "BeginOffset": 7,
            "EndOffset": 16,
            "Score": 0.9999867677688599,
            "Type": "NAME"
        },
        {
            "BeginOffset": 78,
            "EndOffset": 97,
            "Score": 0.9996132254600525,
            "Type": "BANK_ACCOUNT_NUMBER"
        },
        {
            "BeginOffset": 145,
            "EndOffset": 154,
            "Score": 0.9999668002128601,
            "Type": "DATE_TIME"
        },
        {
            "BeginOffset": 257,
            "EndOffset": 267,
            "Score": 0.9999703168869019,
            "Type": "BANK_ACCOUNT_NUMBER"
        },
        {
            "BeginOffset": 292,
            "EndOffset": 301,
            "Score": 0.7057703733444214,
            "Type": "BANK_ACCOUNT_NUMBER"
        },
        {
            "BeginOffset": 340,
            "EndOffset": 374,
            "Score": 0.9999954700469971,
            "Type": "ADDRESS"
        },


Amazon Comprehend returns the PII entity, a confidence score for each of them, and offsets.

Now lets make it a bit more human readable:

In [17]:
detected_pii_entities_df = pd.DataFrame([ [entity['Type'], entity['Score']] for entity in detected_pii_entities['Entities']],
                columns=['Type', 'Score'])

print('This was the text analyzed:')
print(sample_text)
print()
display (detected_pii_entities_df)

This was the text analyzed:

Hello Zhang Wei. Your AnyCompany Financial Services, LLC credit card account 1111-0000-1111-0000 has a minimum payment of $24.53 that is due by July 31st. Based on your autopay settings, we will withdraw your payment on the due date from your bank account XXXXXX1111 with the routing number XXXXX0000. 
Your latest statement was mailed to 100 Main Street, Anytown, WA 98121. 
After your payment is received, you will receive a confirmation text message at 206-555-0100. 
If you have questions about your bill, AnyCompany Customer Service is available by phone at 206-555-0199 or email at support@anycompany.com.




,Type,Score
0,NAME,0.999987
1,BANK_ACCOUNT_NUMBER,0.999613
2,DATE_TIME,0.999967
3,BANK_ACCOUNT_NUMBER,0.999970
4,BANK_ACCOUNT_NUMBER,0.705770
5,ADDRESS,0.999995
6,PHONE,0.999897
7,PHONE,0.999940
8,EMAIL,0.999986


### Label text documents with PII

Amazon Comprehend can help you label documents that contain PII. Select "Labels" in the Personally identifiable information (PII) analysis mode.

The API used to extract the PII enties in the document. We used the [ContainsPiiEntities API](https://docs.aws.amazon.com/comprehend/latest/dg/API_ContainsPiiEntities.html).

Amazon Comprehend returns the different PII labels and the related confidence score for each of them, which indicates how confident the model is in this detection. These labels indicate the presence of these types of PII in the document. 

In [18]:
print('Calling ContainsPiiEntities')
detected_pii_labels = comprehend.contains_pii_entities(Text=sample_text, LanguageCode='en')
print(json.dumps(detected_pii_labels, sort_keys=True, indent=4))
print('End of ContainsPiiEntities\n')

Calling ContainsPiiEntities
{
    "Labels": [
        {
            "Name": "EMAIL",
            "Score": 1.0
        },
        {
            "Name": "DATE_TIME",
            "Score": 1.0
        },
        {
            "Name": "BANK_ROUTING",
            "Score": 0.8962613940238953
        },
        {
            "Name": "NAME",
            "Score": 0.7186043858528137
        },
        {
            "Name": "PHONE",
            "Score": 1.0
        }
    ],
    "ResponseMetadata": {
        "HTTPHeaders": {
            "content-length": "197",
            "content-type": "application/x-amz-json-1.1",
            "date": "Wed, 25 May 2022 13:01:43 GMT",
            "x-amzn-requestid": "139e6509-500b-4ca3-86f7-69170b718f1f"
        },
        "HTTPStatusCode": 200,
        "RequestId": "139e6509-500b-4ca3-86f7-69170b718f1f",
        "RetryAttempts": 0
    }
}
End of ContainsPiiEntities



Amazon Comprehend returns the PII entity name and full scores.

Now lets make it a bit more human readable:

In [19]:
detected_pii_labels_df = pd.DataFrame([ [entity['Name'], entity['Score']] for entity in detected_pii_labels['Labels']],
                columns=['Name', 'Score'])

print('This was the text analyzed:')
print(sample_text)
print()
display (detected_pii_labels_df)

This was the text analyzed:

Hello Zhang Wei. Your AnyCompany Financial Services, LLC credit card account 1111-0000-1111-0000 has a minimum payment of $24.53 that is due by July 31st. Based on your autopay settings, we will withdraw your payment on the due date from your bank account XXXXXX1111 with the routing number XXXXX0000. 
Your latest statement was mailed to 100 Main Street, Anytown, WA 98121. 
After your payment is received, you will receive a confirmation text message at 206-555-0100. 
If you have questions about your bill, AnyCompany Customer Service is available by phone at 206-555-0199 or email at support@anycompany.com.




,Name,Score
0,EMAIL,1.000000
1,DATE_TIME,1.000000
2,BANK_ROUTING,0.896261
3,NAME,0.718604
4,PHONE,1.000000


## <a name="conclusion"></a>Conclusion

You have now learned how to use the pre-trained APIs using the Python SDK.

For examples of how to use these APIs from the AWS Management Console, follow the steps in "Using the AWS Management Console" in the workshop website.

## <a name="clean-up"></a>Clean Up

Once you have finished using this notebook, make sure to stop and delete this Amazon SageMaker Notebook instance in the [Amazon SageMaker Console](https://console.aws.amazon.com/sagemaker/) to avoid incurring additional costs.